# HarvestStat Data Profiling - Madagascar

In [1]:
from functools import reduce
import glob, json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_npz, PrintAdminUnits, PlotAdminShapes
from tools import FDW_PD_Sweeper, FDW_PD_AvalTable, FDW_PD_Compiling, FDW_PD_ValidateFnidName
from tools import FDW_PD_CreateAdminLink, FDW_PD_RatioAdminLink, FDW_PD_ConnectAdminLink
from tools import FDW_PD_CaliSeasonYear
from tools_graphic import PlotBarProduction, PlotLinePAY
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

In [2]:
# Load FEWS NET administrative boundaries ------- #
epsg = 'epsg:32739' # Madagascar
fn_shapes = sorted(glob.glob('../data/shapefile/fewsnet/MG_Admin?_????.shp'))
shape_all = []
for fn in fn_shapes:
    name = fn[-18:-4]
    exec('%s = gpd.read_file("%s").to_crs("%s")' % (name, fn, epsg))
    exec('%s["area"] = %s["geometry"].area/10**6' % (name, name))
    exec('shape_all.append(%s)' % (name))
shape_all = pd.concat(shape_all, axis=0).reset_index(drop=True)
PrintAdminUnits(shape_all)
# ----------------------------------------------- #

# FDW API host address -------------------------- #
host = 'https://fdw.fews.net'
auth = tuple(json.loads(open('token.json', "r").read()))
parameters = {
    'format': 'json',
    'country': 'Madagascar',
    'product': ['R011','R012','R013','R014','R015','R017','R018'],
    'survey_type': 'crop:best'
}
endpoint = '/api/cropproductionindicatorvalue/'
response = requests.get(host + endpoint, auth=auth, params=parameters, proxies={})
response.raise_for_status()
df = pd.DataFrame.from_records(response.json())
# ----------------------------------------------- #

- FEWS NET admin shapefiles ------------------- #
| year	 | Admin1   | # units   | Admin2   | # units   | Admin3   | # units   |
| 1982	 | MG1982A1 | 6	| MG1982A2	| 29	| MG1982A3	| 110	|
| 2004	 | MG2004A1 | 6	| MG2004A2	| 22	| MG2004A3	| 111	|
| 2009	 | MG2009A1 | 6	| MG2009A2	| 22	| MG2009A3	| 119	|
----------------------------------------------- #


In [3]:
# Manual Pre-processing before Sweeping --------- #
# 1. Default setting 
# a) None-type crop production system
df.loc[df['crop_production_system'].isna(), 'crop_production_system'] = 'none'
df.loc[df['crop_production_system'] == '', 'crop_production_system'] = 'none'
# b) None-type population group
df.loc[df['population_group'].isna(), 'population_group'] = 'none'
df.loc[df['population_group'] == '', 'population_group'] = 'none'
# 2. Manual setting
# a) Remove reporting units
df = df[df['fnid'] != 'MG1982R3010101']
# ----------------------------------------------- #

# FDW Production Data Inspection ---------------- #
df, df_raw = FDW_PD_Sweeper(df, area_priority='Area Planted')
table_dict = FDW_PD_AvalTable(df, shape_all)
# ----------------------------------------------- #

# FEWS NET Shapefile comparison ----------------- #
shape_used = pd.concat([
    MG_Admin3_1982, MG_Admin3_2004, MG_Admin2_2009
], axis=0)
PlotAdminShapes(shape_used, label=True)
# ----------------------------------------------- #

- Remove missing records ---------------------- #
Orignial data points: 47,349
Removed 4,160 "Missing Value" points
13,961/15,585 "Area Planted" points are retained.
15,418/15,585 "Quantity Produced" points are retained.
13,810/15,585 "Yield" points are retained.
0/594 "Area Harvested" points are retained.
Current data points: 43,189

- Minor changes are applied ------------------- #

- Basic information --------------------------- #
Data period: 1990 - 2020
9 grain types are found: Beans (mixed), Cassava, Groundnuts (In Shell), Maize (Corn), Pigeon Peas, Potato (Irish), Rice (Paddy), Sugarcane (for sugar), Sweet Potatoes
1 seasons are found: Annual (05-01)
1 crop production system are found: none
Data sources include:
[1] Ministry of Agriculture, Madagascar --- Annuaire des Statistiques Agricoles, Madagascar
[2] Ministry of Agriculture, Madagascar --- FEWS NET Agro Maps, Madagascar
[3] Ministry of Agriculture, Madagascar --- Official agricultural statistics, Madagascar
Administrative-

- Madagascar crop seasonal calendar

![FEWS NET](https://fews.net/sites/default/files/styles/large_width_880/public/2023-03/seasonal-calendar-madagascar.png?itok=xj7JlAs0)

<img src="https://www.arcgis.com/sharing/rest/content/items/7089db1e20d140d59f8db79afb0996ff/data"  width="60%">

- FDW data consists of `MG1982A3`, `MG1982R3`, `MG2004A3`, and `MG2009A2`

| Year  | Admin-1  | # units  | Admin-2  | # units | Admin-3  | # units |
| :---: | :----:   | :----:   | :----:   | :---:   | :----:   | :---:   |
| 1982	 | MG1982A1 | 6	| MG1982A2	| 29	| MG1982A3	| 110	|
| 2004	 | MG2004A1 | 6	| MG2004A2	| 22	| MG2004A3	| 111	|
| 2009	 | MG2009A1 | 6	| **`MG2009A2`**	| 22	| MG2009A3	| 119	|

- In 2004, 1 district is divided and added.

| 1982-2004 | 2004-2009 |
| :---:| :---:|
|Ambositra (MG1982A3030102) | Ambositra (MG2004A3030202), Manandriana (MG2004A3030204) |

- Comparison between admin boundaries.

![image](https://github.com/chc-ucsb/gscd/blob/main/figures/MG_admin_shapes.png?raw=true)

- In 2009, admin-level 3 data became admin-level 2 data.
- **`MG2009A2`** is used to represent the current admin-level 2 crop data.
- Madagascar has a single crop season: `Annual`.
- Madagascar has no population group(s).

In [4]:
# Define the latest shapefile ------------------- #
latest_level = 2
shape_latest = MG_Admin2_2009.copy().to_crs('epsg:4326')
# ----------------------------------------------- #

# Validation of FNIDs and Names ----------------- #
df = FDW_PD_ValidateFnidName(df, shape_used, shape_latest)
# Manual change
idx = shape_used['FNID'].apply(lambda x: x[6:8] == 'A3')
shape_used.loc[idx, 'name'] = shape_used.loc[idx, 'ADMIN3']
# ----------------------------------------------- #

# FDW Production Data Compiling ----------------- #
area, prod = FDW_PD_Compiling(df, shape_used)
area_all, prod_all = area.copy(), prod.copy()
mdx_pss = area.columns.droplevel([0,1]).unique()
# ----------------------------------------------- #

In [5]:
# Aggregation to lower admin level -------------- #
# *We aggregate admin-3 data to admin-2 data
# Generate AdminLink manually
from tools import CreateLinkAdmin, invert_dicts
link_1982, _, _ = CreateLinkAdmin(MG_Admin2_2009, MG_Admin3_1982, old_on='ADMIN2', new_on='ADMIN3')
link_1982 = invert_dicts(link_1982)
link_2004, _, _ = CreateLinkAdmin(MG_Admin2_2009, MG_Admin3_2004, old_on='ADMIN2', new_on='ADMIN3')
link_2004 = invert_dicts(link_2004)
assert link_1982.keys() == link_2004.keys()
link = {}
for fnid in link_1982.keys():
    link[fnid] = [*link_1982[fnid], *link_2004[fnid]]
# Connect data with AdminLink
area_new = []
prod_new = []
for fnid in link.keys():
    area_merged = [area[i].droplevel(0, axis=1) for i in [fnid, *link[fnid]]]
    area_merged = reduce(lambda a, b: a.add(b, fill_value=0), area_merged)
    area_merged.columns = area[fnid].columns
    area_merged = pd.concat([area_merged], keys=[fnid], names=['fnid'], axis=1)
    area_new.append(area_merged)
    prod_merged = [prod[i].droplevel(0, axis=1) for i in [fnid, *link[fnid]]]
    prod_merged = reduce(lambda a, b: a.add(b, fill_value=0), prod_merged)
    prod_merged.columns = prod[fnid].columns
    prod_merged = pd.concat([prod_merged], keys=[fnid], names=['fnid'], axis=1)
    prod_new.append(prod_merged)
area_new = pd.concat(area_new, axis=1)
prod_new = pd.concat(prod_new, axis=1)
assert area.sum().sum() == area_new.sum().sum()
assert prod.sum().sum() == prod_new.sum().sum()
# ----------------------------------------------- #

In [6]:
# Manual correction ----------------------------- #
crop_new = prod_new/area_new
# area_new.loc[2004,pd.IndexSlice['MG2009A20501',:,'Maize (Corn)','Annual',:]] = np.nan
# crop_new.loc[2004,pd.IndexSlice['MG2009A20501',:,'Maize (Corn)','Annual',:]] = np.nan
# ----------------------------------------------- #

# Complete long format DataFrame ---------------- #
df_area = area_new.T.stack().rename('value').reset_index()
df_area['indicator'] = 'area'
df_prod = prod_new.T.stack().rename('value').reset_index()
df_prod['indicator'] = 'production'
df_yield = crop_new.T.stack().rename('value').reset_index()
df_yield['indicator'] = 'yield'
stack = pd.concat([df_area, df_prod, df_yield], axis=0)
# Add "planting year"
cols = ['season_name','product','crop_production_system','planting_month','harvest_year','harvest_month','planting_year']
season_table = df[cols].drop_duplicates()
stack = stack.merge(season_table, on=cols[:-1])
# Add country and admin names
stack = stack.merge(df[['fnid','country','country_code','admin_1','admin_2']].drop_duplicates(), on='fnid', how='inner')
names = [
    'fnid','country','country_code','admin_1','admin_2','name',
    'product','season_name','planting_year','planting_month','harvest_year','harvest_month',
    'crop_production_system','indicator','value'
]
stack = stack[names]
# ----------------------------------------------- #

# Final Processing ------------------------------ #
# Manual change of product name
product_category_custom = {
    'Beans (mixed)': 'Beans',
    'Cassava': 'Cassava',
    'Groundnuts (In Shell)': 'Groundnuts',
    'Maize (Corn)': 'Maize',
    'Pigeon Peas': 'Pigeon Peas',
    'Potato (Irish)': 'Potato',
    'Rice (Paddy)': 'Rice',
    'Sugarcane (for sugar)': 'Sugarcane',
    'Sweet Potatoes': 'Sweet Potatoes'
}
stack['product'] = stack['product'].replace(product_category_custom)
# Calibration of crop calendar using External Crop Calendar (ECC)
ecc = pd.read_csv('../data/crop_calendar/external_crop_calendar.csv')
ecc = ecc[ecc['country'] == 'Madagascar']
stack, link_ratio = FDW_PD_CaliSeasonYear(stack, ecc, link_ratio=None)
# None admin names
stack.loc[stack['admin_1'].isna(), 'admin_1'] = 'none'
stack.loc[stack['admin_2'].isna(), 'admin_2'] = 'none'
# ----------------------------------------------- #

# Save data
fn_out = '../data/crop/adm_crop_production_MG.csv'
stack.to_csv(fn_out); print(f'{fn_out} is saved.')

All [season_name, product, crop_production_system] are in the external crop calendar.
../data/crop/adm_crop_production_MG.csv is saved.


## Visualization of production data

In [7]:
# Bar chart of national crop production
country_iso, country_name = 'MG', 'Madagascar'
df = pd.read_csv('../data/crop/adm_crop_production_%s.csv' % country_iso, index_col=0)
df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_order = ['Rice','Maize']
for season_name in ['Annual']:
    footnote = 'National crop production in %s - %s' % (country_name, season_name)
    fn_save = '../figures/%s_bar_natgrainprod_%s.png' % (country_iso, season_name)
    sub = df[df['season_name'] == season_name]
    fig = PlotBarProduction(sub, year, product_order, footnote, fn_save)
    fig.show()

../figures/MG_bar_natgrainprod_Annual.png is saved.


In [8]:
# Lineplot of Production-Area-Yield time-series
country_iso, country_name = 'MG', 'Madagascar'
df = pd.read_csv('../data/crop/adm_crop_production_%s.csv' % country_iso, index_col=0)
df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_season = [
    ['Rice','Annual'],
    ['Cassava','Annual'],
    ['Maize','Annual'],
]
for product_name, season_name in product_season:
    footnote = 'Production-Area-Yield time-series of %s - %s - %s' % (country_iso, product_name, season_name)
    fn_save = '../figures/%s_line_pay_%s_%s.png' % (country_iso, product_name, season_name)
    sub = df[(df['product'] == product_name) & (df['season_name'] == season_name)]
    fig = PlotLinePAY(sub, year, footnote, fn_save)
    fig.show()

../figures/MG_line_pay_Rice_Annual.png is saved.


../figures/MG_line_pay_Cassava_Annual.png is saved.


../figures/MG_line_pay_Maize_Annual.png is saved.
